<a href="https://colab.research.google.com/github/Rajitha-SL/My-AI-Projects/blob/AI-and-ML-learning/Udacity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an example for a typical Convolutional Block

In [2]:
# self.conv1 = nn.Conv2d(3 ,16, 3, padding=1)
#